In [97]:
import xarray as xr
import zipfile
import re
import io

from datetime import datetime, date

path = "../data/raw/ka.zip"

In [98]:
with zipfile.ZipFile(path, "r") as zip_file:
    file_list = zip_file.namelist()

In [99]:
# if the date has no day, the special character is M and that means that it´s the monthly average.

""" 
<zip_name>/<frequency>/GW1AM"_YYYYmmdd_01< D: dayly / M: monthly>_<Ascending / Descending pass>_L3SGT36LA2220220_corrected.h5

We need to extract: 
    dayly and monthly xarrays

"""

# Regex for extracting this information
isday_regex = r"_01D_"
date_regex = r"_(\d{4})(\d{2})(\d{2})_"
pass_regex = r"_EQM([AD])_"


# First, separate the daily and monthly files
with zipfile.ZipFile(path, "r") as zip_file:
    file_list = zip_file.namelist()

    day_files = []
    month_files = []

    for file_name in file_list:
        # Check and separate the monthly and daily files:
        is_day = re.findall(pattern=isday_regex, string=file_name)

        if is_day:
            day_files.append(file_name)
        else:
            month_files.append(file_name)

    # Now we call the same function to load an xarray for each list


file_list
[]

In [5]:
print(len(day_files))
print(len(month_files))

730
24


In [77]:
class ZipLoader:
    def __init__(self, path):
        self.path = path
        # Separate daily data from monthly averages
        self.day_files, self.month_files = self.SeparateDayMonth()

        # Create the empty xarrays
        self.coords = self.GetCoordinates()

        self.daily_data = xr.Dataset(coords=self.coords)
        self.montly_data = xr.Dataset(coords=self.coords)

    def __str__(self) -> str:
        response = (
            f"Loading {self.path}"
            f"\n Daily files {len(self.day_files)}"
            f"\n Monthly files {len(self.month_files)}"
            f"\n Coordinates: {self.coords.sizes} \n"
        )
        return response

    def SeparateDayMonth(self):
        with zipfile.ZipFile(self.path, "r") as zip_file:
            file_list = zip_file.namelist()
            day_files = []
            month_files = []

            for file_name in file_list:
                # Check and separate the monthly and daily files:
                is_day = re.findall(pattern=isday_regex, string=file_name)

                if is_day:
                    day_files.append(file_name)

                elif file_name.endswith(".h5"):  # Avoid the main folder
                    month_files.append(file_name)

        return day_files, month_files

    def GetCoordinates(self):
        """
        Returns the latitude and longitude shapes of the dataset
        """

        # Load the first day file to get the size of the lat-lon dimensions
        with zipfile.ZipFile(self.path, "r") as zip_file:
            with zip_file.open(self.day_files[0]) as h5_file:
                h5_data = h5_file.read()
                h5_data_io = io.BytesIO(h5_data)
                h5_xarray = xr.open_dataset(h5_data_io)

            return h5_xarray.coords

    def LoadDailyData(self):
        """
        Populate the empty xarray with the data and add the time dimention
        """
        times = []
        for file_name in self.day_files:
            time = self.GetTime(file_name)
            times.append(time)

        # Modify the xarray and add the new time dimention

        self.daily_data.assign_coords()

        pass

    def GetTime(self, file_name):
        """
        Given a filename, return its date
        """
        date_regex = r"_(\d{4})(\d{2})(\d{2})_"
        year, month, day = [
            int(value) for value in re.findall(date_regex, file_name)[0]
        ]

        if day == 0:
            return datetime(year, month, 1)

        return datetime(year, month, day)

In [96]:
date_regex = r"_(\d{4})(\d{2})(\d{2})_"

file_name = "ka_sample/36ghz/GW1AM2_20170100_01M_EQMA_L3SGT36LA2220220_corrected.h5"

year, month, day = [int(value) for value in re.findall(date_regex, file_name)[0]]

date(year, month, day=1)

datetime.date(2017, 1, 1)

In [78]:
# path = "../data/raw/ka.zip"
loader = ZipLoader(path)
print(loader)

Loading ../data/raw/ka.zip
 Daily files 730
 Monthly files 24
 Coordinates: Frozen({'lon': 1388, 'lat': 584}) 



In [72]:
with zipfile.ZipFile(path, "r") as zip_file:
    file_list = zip_file.namelist()
    with zip_file.open(file_list[1]) as h5_file:
        # Read the HDF5 file data
        h5_data = h5_file.read()

        # Load the HDF5 file data into an Xarray dataset
        h5_data_io = io.BytesIO(h5_data)
        h5_xarray = xr.open_dataset(h5_data_io)
        coords = h5_xarray.coords

Frozen({'lon': 1388, 'lat': 584})

'lat'